<a href="https://colab.research.google.com/github/MatSpad/Advanced-Coding-for-Data-Analytics-Project-2024/blob/main/ACfDA_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [44]:
# Importing libraries       (useful for)
import os                   #managing directories
import time                 #checking code run times
from tqdm.auto import tqdm  #generating fancy progress bars
import gzip                 #decompressing and reading .gz files
import networkx as nx       #generating and analyzing graph objects

# Cloning GitHub repository as Colab directory
if os.getcwd()[-17:] == 'cloned-repository':
  os.chdir("..")
! git clone -l -s https://github.com/MatSpad/Advanced-Coding-for-Data-Analytics-Project-2024.git cloned-repository
%cd cloned-repository
#!ls

# Bulding a Pandas dataframe containing all edges as presented by the related file
import pandas as pd
edgesDF = pd.read_csv('Inputs/hetionet-v1.0-edges.sif.gz', compression='gzip', sep='\t')
display(edgesDF)
edgesFileLen =len(edgesDF.index)

'''
# Bulding a NetworkX directed graph by reading the edges file
start = time.time()
G = nx.DiGraph()
with gzip.open('Inputs/hetionet-v1.0-edges.sif.gz','r') as edgesFile:
  edgesFile.readline()
  for line in edgesFile:
    lineElements = str(line)[2:-5].split('\\t')
    G.add_edge(lineElements[0], lineElements[2], edgeType=lineElements[1])
print(G.number_of_edges())
end = time.time()
print(end - start)  #slightly faster than progress-bar-enriched graph building
'''

# Bulding a NetworkX directed graph by reading the edges file (with progress bar)
G = nx.DiGraph()
with gzip.open('Inputs/hetionet-v1.0-edges.sif.gz','r') as edgesFile:
  edgesFile.readline()
  for i in tqdm(range(edgesFileLen)):
    lineElements = str(edgesFile.readline())[2:-5].split('\\t')
    G.add_edge(lineElements[0], lineElements[2], edgeType=lineElements[1])

fatal: destination path 'cloned-repo' already exists and is not an empty directory.
ACfDA_project.ipynb  cloned-repo  Inputs  Outputs  README.md


,source,metaedge,target
0,Gene::9021,GpBP,Biological Process::GO:0071357
1,Gene::51676,GpBP,Biological Process::GO:0098780
2,Gene::19,GpBP,Biological Process::GO:0055088
3,Gene::3176,GpBP,Biological Process::GO:0010243
4,Gene::3039,GpBP,Biological Process::GO:0006898
...,...,...,...
2250192,Anatomy::UBERON:0000057,AeG,Gene::65009
2250193,Anatomy::UBERON:0000474,AeG,Gene::80279
2250194,Anatomy::UBERON:0002048,AeG,Gene::1211
2250195,Anatomy::UBERON:0002048,AeG,Gene::8843


  0%|          | 0/2250197 [00:00<?, ?it/s]